In [10]:
import seaborn as sns
import pandas as pd
import pylab as plt
import numpy as np
from steam import WebAPI
import time
import os
import glob
import json
import tensorflow as tf
print(tf.__version__)
import sys
from IPython import display
import requests
#dont keep api key in github!!!
api_file = os.path.join('..','..','..','apikeys','steam_api_key.txt')
with open(api_file, 'r') as fin:
    api = WebAPI(key=fin.readline())

2.0.0


In [11]:
#declare interface urls
#dota2_beta should be used for testing in order to not aggressively make API requests
dota2_id = '570'
dota2_beta_id = '205790'

hero_list = api.call('IEconDOTA2_'+dota2_id+'.GetHeroes')
hero_vocab=[int(hero_id['id']) for hero_id in hero_list['result']['heroes']]
print(hero_vocab)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 31, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 68, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 106, 107, 109, 110, 111, 105, 112, 113, 108, 114, 120, 119, 121, 129]


# Build TFRecords file
Puts all training data in the training_data folder into one TFRecords file
Do this to nicely handle the features that are numpy arrays (radiant_heroes and dire_heroes)

In [12]:
dota2_df = None
files=glob.glob(os.path.join('Dota_data','mixed_skill','*.csv'))
files.sort(key=os.path.getmtime)
print(files[0])
for filename in files[-1:]:
    if type(dota2_df)==None:
        dota2_df = pd.read_csv(filename, usecols=range(0,23))
    else:
        dota2_df = pd.concat([dota2_df, pd.read_csv(filename, usecols=range(0,23))],ignore_index=True)

dota2_df = dota2_df.reindex(np.random.permutation(dota2_df.index))
print(dota2_df.columns)
dota2_df.describe()

Dota_data/mixed_skill/4725403110-4781085451.csv
Index(['ID', 'radiant_win', 'radiant_xp', 'dire_xp', 'radiant_gold',
       'dire_gold', 'radiant_kills', 'dire_kills', 'radiant_healing',
       'dire_healing', 'radiant_towerdamage', 'dire_towerdamage', 'match_id',
       'radiant_heroes0', 'dire_heroes0', 'radiant_heroes1', 'dire_heroes1',
       'radiant_heroes2', 'dire_heroes2', 'radiant_heroes3', 'dire_heroes3',
       'radiant_heroes4', 'dire_heroes4'],
      dtype='object')


,ID,radiant_win,radiant_xp,dire_xp,radiant_gold,dire_gold,radiant_kills,dire_kills,radiant_healing,dire_healing,...,radiant_heroes0,dire_heroes0,radiant_heroes1,dire_heroes1,radiant_heroes2,dire_heroes2,radiant_heroes3,dire_heroes3,radiant_heroes4,dire_heroes4
count,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000,...,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000,287416.000000
mean,143707.500000,0.547402,108936.501296,108245.521488,71398.804068,70517.470061,30.670370,29.847507,0.002272,3692.235954,...,52.834762,52.900604,52.996876,53.116295,52.971891,53.098888,52.903033,53.014484,52.786247,52.950518
std,82969.996822,0.497749,50081.940580,52053.025385,27735.982901,29633.745523,13.389364,14.395740,0.081167,5060.362360,...,35.699988,35.689077,35.789319,35.749345,35.741674,35.747986,35.753247,35.744772,35.769582,35.694289
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,71853.750000,0.000000,72688.087500,69510.475000,52915.000000,49535.000000,21.000000,19.000000,0.000000,325.000000,...,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
50%,143707.500000,1.000000,105657.850000,105186.833350,69960.000000,69575.000000,31.000000,30.000000,0.000000,1797.000000,...,48.000000,47.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,47.000000,48.000000
75%,215561.250000,1.000000,141294.512500,142534.100025,88110.000000,89235.000000,40.000000,40.000000,0.000000,5111.000000,...,83.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
max,287415.000000,1.000000,542318.266700,507291.733300,291645.000000,266075.000000,112.000000,164.000000,18.000000,88485.000000,...,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000


In [13]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [14]:
def convert_to_tfrecords(filename, examples, targets, target_name):
    '''Convert pandas dataframe to TFRecord. The format of the heroes
          is reindexed by the hero_vocab so that the values are equal
          to indices
    '''
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename, options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    target_array = np.array(targets[target_name])
    
    counter=0
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 5000 games
        if not i % 5000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the arrays
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        target = target_array[i]
        
        # Create a feature
        if target_name=='radiant_win':
            feature = {'radiant_heroes': _int64_list_feature(rad_heroes),
                   'dire_heroes': _int64_list_feature(dire_heroes),
                   'label': _int64_feature(target)}
        else:
            feature = {'radiant_heroes': _int64_list_feature(rad_heroes),
                   'dire_heroes': _int64_list_feature(dire_heroes),
                   'label': _float_feature(target)}
        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    print('heroes not in vocab',counter)
    writer.close()
    sys.stdout.flush()

In [15]:
def convert_to_tfrecords_novocab(filename, examples, targets, target_name):
    '''Convert pandas dataframe to TFRecord. The format of the heroes
          is reindexed by the hero_vocab so that the values are equal
          to indices
    '''
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename, options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    target_array = np.array(targets[target_name])
    
    counter=0
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 5000 games
        if not i % 5000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the arrays
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        
        rad_heroes_index=[]
        dire_heroes_index=[]
        pass_outer=False
        for j in range(5):
            #add heroes to the heroes by index list for this game
            try:
                dire_heroes_index.append(hero_vocab.index(dire_heroes[j]))
                rad_heroes_index.append(hero_vocab.index(rad_heroes[j]))
            #if the hero isnt in the vocab then break and pass so 
            #  the example isnt used for training
            except ValueError:
                counter+=1
                pass_outer = True
                break
        if pass_outer:
            pass_outer=False
            continue

        target = target_array[i]
        
        # Create a feature
        if target_name=='radiant_win':
            feature = {'radiant_heroes': _int64_list_feature(rad_heroes_index),
                   'dire_heroes': _int64_list_feature(dire_heroes_index),
                   'label': _int64_feature(target)}
        else:
            feature = {'radiant_heroes': _int64_list_feature(rad_heroes_index),
                   'dire_heroes': _int64_list_feature(dire_heroes_index),
                   'label': _float_feature(target)}
        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    print('heroes not in vocab',counter)
    writer.close()
    sys.stdout.flush()

In [16]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_dual(filename, examples, target_df):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    win_array = np.array(target_df['radiant_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    
    counter=0
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        
        radiant_win = win_array[i]
        gold_diff = gold_diff_array[i]
        
        # Create a feature
        feature_description = {'radiant_heroes': _int64_list_feature(rad_heroes),
                   'dire_heroes': _int64_list_feature(dire_heroes),
                   'radiant_win': _int64_feature(radiant_win),
                   'gold_diff': _float_feature(gold_diff)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature_description))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    print('heroes not in vocab',counter)
    writer.close()
    sys.stdout.flush()

In [17]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_dual_novocab(filename, examples, target_df):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    win_array = np.array(target_df['radiant_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    
    counter=0
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        
        rad_heroes_index=[]
        dire_heroes_index=[]
        pass_outer=False
        for j in range(5):
            #add heroes to the heroes by index list for this game
            try:
                dire_heroes_index.append(hero_vocab.index(dire_heroes[j]))
                rad_heroes_index.append(hero_vocab.index(rad_heroes[j]))
            #if the hero isnt in the vocab then break and pass so 
            #  the example isnt used for training
            except ValueError:
                counter+=1
                pass_outer = True
                break
        if pass_outer:
            pass_outer=False
            continue
        
        radiant_win = win_array[i]
        gold_diff = gold_diff_array[i]
        
        # Create a feature
        feature_description = {'radiant_heroes': _int64_list_feature(rad_heroes_index),
                   'dire_heroes': _int64_list_feature(dire_heroes_index),
                   'radiant_win': _int64_feature(radiant_win),
                   'gold_diff': _float_feature(gold_diff)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature_description))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    print('heroes not in vocab',counter)
    writer.close()
    sys.stdout.flush()

In [18]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_triple_novocab(filename, examples, target_df):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    win_array = np.array(target_df['radiant_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    total_gold_array = np.array(target_df['total_gold'])
    
    counter=0
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        
        rad_heroes_index=[]
        dire_heroes_index=[]
        pass_outer=False
        for j in range(5):
            #add heroes to the heroes by index list for this game
            try:
                dire_heroes_index.append(hero_vocab.index(dire_heroes[j]))
                rad_heroes_index.append(hero_vocab.index(rad_heroes[j]))
            #if the hero isnt in the vocab then break and pass so 
            #  the example isnt used for training
            except ValueError:
                counter+=1
                pass_outer = True
                break
        if pass_outer:
            pass_outer=False
            continue
        
        radiant_win = win_array[i]
        gold_diff = gold_diff_array[i]
        total_gold = total_gold_array[i]
        
        # Create a feature
        feature_description = {'radiant_heroes': _int64_list_feature(rad_heroes_index),
                   'dire_heroes': _int64_list_feature(dire_heroes_index),
                   'radiant_win': _int64_feature(radiant_win),
                   'gold_diff': _float_feature(gold_diff),
                   'total_gold': _float_feature(total_gold)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature_description))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    print('heroes not in vocab',counter)
    writer.close()
    sys.stdout.flush()

In [34]:
#target_name to name tfrecords file and the target_df column to call
#for now don't keep all possible targets in target_df
#target_df selection is made with the preprocess_target function call
def convert_to_tfrecords_penta_novocab(filename, examples, target_df):
    # open the TFRecords file
    options = tf.io.TFRecordOptions('GZIP')
    writer = tf.io.TFRecordWriter(filename,options=options)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    win_array = np.array(target_df['radiant_win'])
    gold_diff_array = np.array(target_df['gold_diff'])
    total_gold_array = np.array(target_df['total_gold'])
    towdam_diff_array = np.array(target_df['towdam_diff'])
    total_towdam_array = np.array(target_df['total_towdam'])
    
    counter=0
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 10000 games
        if not i % 10000:
            print('Data progress: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the hero_array
        rad_heroes = radiant_hero_array[:][i]
        dire_heroes = dire_hero_array[:][i]
        
        rad_heroes_index=[]
        dire_heroes_index=[]
        pass_outer=False
        for j in range(5):
            #add heroes to the heroes by index list for this game
            try:
                dire_heroes_index.append(hero_vocab.index(dire_heroes[j]))
                rad_heroes_index.append(hero_vocab.index(rad_heroes[j]))
            #if the hero isnt in the vocab then break and pass so 
            #  the example isnt used for training
            except ValueError:
                counter+=1
                pass_outer = True
                break
        if pass_outer:
            pass_outer=False
            continue
        
        radiant_win = win_array[i]
        gold_diff = gold_diff_array[i]
        total_gold = total_gold_array[i]
        towdam_diff = towdam_diff_array[i]
        total_towdam = total_towdam_array[i]
    
        # Create a feature
        feature_description = {'radiant_heroes': _int64_list_feature(rad_heroes_index),
                   'dire_heroes': _int64_list_feature(dire_heroes_index),
                   'radiant_win': _int64_feature(radiant_win),
                   'gold_diff': _float_feature(gold_diff),
                   'total_gold': _float_feature(total_gold),
                   'tower_damage_diff': _float_feature(towdam_diff),
                   'total_tower_damage': _float_feature(total_towdam)}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature_description))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    print('heroes not in vocab',counter)
    writer.close()
    sys.stdout.flush()

In [30]:
def preprocess_features(dota2_df):
    """Take dota2_df and create a dataframe containing only the features for our model
            Args: 
                dota2_df: Dataframe containing dota2 training and test data
            returns: 
                processed_df: pandas DataFrame containing only feature columns
    """
    
    #Use arrays of heroes for each team as features rather than each hero
    #individually to help model fit?
    #This is an attempt to make the model more similar to the movie review
    #text analysis example in the Google ML Crash Course
    processed_df = pd.DataFrame()
    
    radiant_heroes = list(np.array(dota2_df.loc[:,['radiant_heroes0',
                                                   'radiant_heroes1',
                                                   'radiant_heroes2',
                                                   'radiant_heroes3',
                                                   'radiant_heroes4']]).astype(int))
    processed_df['radiant_heroes'] = radiant_heroes
    
    dire_heroes = list(np.array(dota2_df.loc[:,['dire_heroes0',
                                                'dire_heroes1',
                                                'dire_heroes2',
                                                'dire_heroes3',
                                                'dire_heroes4']]).astype(int))        
    processed_df['dire_heroes'] = dire_heroes
     
    return processed_df
    
def preprocess_targets_win(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model.
      (radiant_win here)
            Args: 
                dota2_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column
    """
    target_df = pd.DataFrame()
    target_df['radiant_win'] = dota2_df['radiant_win']
    
    return target_df

def preprocess_targets_gold(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                dota2_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['gold_diff'] = (dota2_df['radiant_gold']-dota2_df['dire_gold'])\
                                /(dota2_df['radiant_gold']+dota2_df['dire_gold'])
    target_df['gold_diff'] = (target_df['gold_diff']-target_df['gold_diff'].mean())\
                                /target_df['gold_diff'].std()
    target_df['gold_diff'] = target_df['gold_diff'].fillna(0)

    return target_df

def z_score(column):
    return (column-column.mean())/column.std()

def preprocess_targets_dual_gold_win(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                dota2_df: Dataframe containing dota training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['radiant_win'] = dota2_df['radiant_win']
    target_df['gold_diff'] = dota2_df['radiant_gold']-dota2_df['dire_gold']
    target_df['gold_diff'] = z_score(target_df['gold_diff'])
    target_df['gold_diff'] = target_df['gold_diff'].fillna(0)
    
    return target_df

def preprocess_targets_triple_gold_win(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                dota2_df: Dataframe containing dota training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['radiant_win'] = dota2_df['radiant_win']
    target_df['gold_diff'] = dota2_df['radiant_gold']-dota2_df['dire_gold']
    target_df['gold_diff'] = z_score(target_df['gold_diff'])
    target_df['gold_diff'] = target_df['gold_diff'].fillna(0)
    
    target_df['total_gold'] = dota2_df['radiant_gold']+dota2_df['dire_gold']
    target_df['total_gold'] = z_score(target_df['total_gold'])
    target_df['total_gold'] = target_df['total_gold'].fillna(0)
    
    return target_df

def preprocess_targets_penta_gold_win_towdam(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model 
        (normalized gold difference in this case)
            Args: 
                dota2_df: Dataframe containing dota training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (gold_diff)
    """
    target_df = pd.DataFrame()
    target_df['radiant_win'] = dota2_df['radiant_win']
    target_df['gold_diff'] = dota2_df['radiant_gold']-dota2_df['dire_gold']
    target_df['gold_diff'] = z_score(target_df['gold_diff'])
    target_df['gold_diff'] = target_df['gold_diff'].fillna(0)
    
    target_df['total_gold'] = dota2_df['radiant_gold']+dota2_df['dire_gold']
    target_df['total_gold'] = z_score(target_df['total_gold'])
    target_df['total_gold'] = target_df['total_gold'].fillna(0)
    
    target_df['towdam_diff'] = dota2_df['radiant_towerdamage']-dota2_df['dire_towerdamage']
    target_df['towdam_diff'] = z_score(target_df['towdam_diff'])
    target_df['towdam_diff'] = target_df['towdam_diff'].fillna(0)
    
    target_df['total_towdam'] = dota2_df['radiant_towerdamage']+dota2_df['dire_towerdamage']
    target_df['total_towdam'] = z_score(target_df['total_towdam'])
    target_df['total_towdam'] = target_df['total_towdam'].fillna(0)
    return target_df

def preprocess_targets_xp(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model 
        (normalized xp difference in this case)
            Args: 
                dota2_df: Dataframe containing lol training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column (xp_diff)
    """
    target_df = pd.DataFrame()
    target_df['xp_diff'] = (dota2_df['radiant_xp']-dota2_df['dire_xp'])/(dota2_df['radiant_xp']+dota2_df['dire_xp'])
    target_df['xp_diff'] = (target_df['xp_diff']-target_df['xp_diff'].mean())/target_df['xp_diff'].std()
    return target_df

### Visualize data for each set

In [21]:
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
test_len = int((len(dota2_df) - train_len)/1.5)
validation_len = len(dota2_df) - train_len - test_len

plotting_df = dota2_df

display.display(plotting_df.iloc[:train_len,:].describe())
display.display(plotting_df.iloc[train_len:train_len+test_len,:].describe())
display.display(plotting_df.iloc[train_len+test_len:,:].describe())


,ID,radiant_win,radiant_xp,dire_xp,radiant_gold,dire_gold,radiant_kills,dire_kills,radiant_healing,dire_healing,...,radiant_heroes0,dire_heroes0,radiant_heroes1,dire_heroes1,radiant_heroes2,dire_heroes2,radiant_heroes3,dire_heroes3,radiant_heroes4,dire_heroes4
count,201191.000000,201191.000000,201191.000000,201191.000000,201191.000000,201191.000000,201191.000000,201191.000000,201191.000000,201191.000000,...,201191.000000,201191.000000,201191.000000,201191.000000,201191.00000,201191.00000,201191.000000,201191.000000,201191.000000,201191.000000
mean,143909.421490,0.546893,109071.027168,108410.273208,71465.045156,70609.383173,30.694136,29.888410,0.002321,3688.861241,...,52.776516,52.910925,53.009354,53.139335,52.96813,53.10916,52.988315,52.988667,52.785338,52.852931
std,83054.748699,0.497797,50124.619206,52102.547408,27767.366122,29664.673491,13.397669,14.387616,0.084305,5043.926209,...,35.709391,35.699423,35.764925,35.768642,35.75231,35.73507,35.811107,35.748058,35.742649,35.686429
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.000000,1.000000,1.000000,1.000000
25%,72019.500000,0.000000,72799.658335,69618.725000,52925.000000,49630.000000,21.000000,19.000000,0.000000,321.000000,...,21.000000,21.000000,21.000000,21.000000,21.00000,21.00000,21.000000,21.000000,21.000000,21.000000
50%,144038.000000,1.000000,105841.633300,105425.233300,70050.000000,69670.000000,31.000000,30.000000,0.000000,1800.000000,...,47.000000,47.000000,48.000000,48.000000,48.00000,48.00000,48.000000,48.000000,47.000000,48.000000
75%,216006.000000,1.000000,141481.466650,142742.516650,88190.000000,89345.000000,40.000000,40.000000,0.000000,5108.000000,...,83.000000,84.000000,84.000000,84.000000,84.00000,84.00000,84.000000,84.000000,84.000000,83.000000
max,287414.000000,1.000000,472236.150000,454558.550000,232250.000000,253705.000000,112.000000,164.000000,18.000000,78674.000000,...,129.000000,129.000000,129.000000,129.000000,129.00000,129.00000,129.000000,129.000000,129.000000,129.000000


,ID,radiant_win,radiant_xp,dire_xp,radiant_gold,dire_gold,radiant_kills,dire_kills,radiant_healing,dire_healing,...,radiant_heroes0,dire_heroes0,radiant_heroes1,dire_heroes1,radiant_heroes2,dire_heroes2,radiant_heroes3,dire_heroes3,radiant_heroes4,dire_heroes4
count,57483.000000,57483.00000,57483.000000,57483.000000,57483.000000,57483.000000,57483.000000,57483.000000,57483.000000,57483.000000,...,57483.000000,57483.000000,57483.000000,57483.000000,57483.000000,57483.000000,57483.000000,57483.000000,57483.000000,57483.000000
mean,143000.997338,0.54938,108514.966075,107755.914757,71207.215786,70262.282588,30.602248,29.753475,0.002262,3696.649218,...,53.133726,52.801507,53.018510,53.048414,52.852113,53.085504,52.680584,52.994016,52.748256,53.068177
std,82670.802783,0.49756,49774.113442,51748.470133,27577.069533,29459.501328,13.354520,14.414953,0.076875,5137.035618,...,35.709997,35.642422,35.903762,35.653269,35.741107,35.786053,35.691965,35.734465,35.774901,35.666166
min,6.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,71566.500000,0.00000,72501.966665,69254.200000,52875.000000,49270.000000,21.000000,19.000000,0.000000,330.000000,...,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
50%,142599.000000,1.00000,105000.600000,104549.050000,69725.000000,69445.000000,31.000000,30.000000,0.000000,1761.000000,...,48.000000,47.000000,48.000000,48.000000,47.000000,48.000000,47.000000,48.000000,47.000000,48.000000
75%,214133.500000,1.00000,140691.283350,142028.533350,87880.000000,88985.000000,40.000000,40.000000,0.000000,5093.000000,...,84.000000,83.000000,84.000000,84.000000,83.000000,84.000000,83.000000,84.000000,83.000000,84.000000
max,287415.000000,1.00000,542318.266700,507291.733300,291645.000000,264420.000000,110.000000,103.000000,9.000000,86116.000000,...,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000


,ID,radiant_win,radiant_xp,dire_xp,radiant_gold,dire_gold,radiant_kills,dire_kills,radiant_healing,dire_healing,...,radiant_heroes0,dire_heroes0,radiant_heroes1,dire_heroes1,radiant_heroes2,dire_heroes2,radiant_heroes3,dire_heroes3,radiant_heroes4,dire_heroes4
count,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000,...,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000,28742.000000
mean,143707.051388,0.547004,108837.890009,108071.473079,71318.293264,70384.453935,30.640248,29.749252,0.001948,3707.032218,...,52.644562,53.026547,52.866258,53.090773,53.237771,53.053754,52.750957,53.236135,52.868590,53.398302
std,82968.881753,0.497794,50392.689206,52307.995003,27831.894221,29761.857623,13.400743,14.413269,0.065655,5020.709385,...,35.611921,35.710624,35.731701,35.807128,35.668113,35.763410,35.468032,35.742855,35.948076,35.802474
min,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,71408.750000,0.000000,72393.408332,69177.366667,52966.250000,49395.000000,21.000000,19.000000,0.000000,330.000000,...,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
50%,143692.000000,1.000000,105617.516650,104835.550000,69845.000000,69172.500000,31.000000,30.000000,0.000000,1836.000000,...,47.000000,47.000000,48.000000,48.000000,48.000000,48.000000,47.000000,48.000000,48.000000,48.000000
75%,215559.000000,1.000000,141203.320825,142125.208325,88043.750000,88980.000000,40.000000,40.000000,0.000000,5149.500000,...,83.000000,84.000000,84.000000,84.000000,84.000000,84.000000,83.000000,84.000000,84.000000,84.000000
max,287413.000000,1.000000,491487.733300,439952.333300,234705.000000,266075.000000,94.000000,92.000000,6.000000,88485.000000,...,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000,129.000000


## Make tfrecords files

### SIngle target tfrecord making

In [22]:
print(train_targets.isnull().sum(),
      validation_targets.isnull().sum(),
      test_targets.isnull().sum())

NameError: name 'train_targets' is not defined

In [23]:
#create tfrecord that needs a vocab for use in feature column models
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
test_len = int((len(dota2_df) - train_len)/1.5)
validation_len = len(dota2_df) - train_len - test_len


target_name='radiant_win'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len+test_len:,:])

if target_name=='radiant_win':
    train_targets = preprocess_targets_win(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_win(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_win(dota2_df.iloc[train_len+test_len:,:])
elif target_name=='gold_diff':
    train_targets = preprocess_targets_gold(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_gold(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_gold(dota2_df.iloc[train_len+test_len:,:])
elif target_name=='xp_diff':
    train_targets = preprocess_targets_xp(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_xp(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_xp(dota2_df.iloc[train_len+test_len:,:])

convert_to_tfrecords(os.path.join('Dota_data','mixed_skill','dota2_training_data',
                     target_name+'dota2_training_data.tfrecords'),
                     train_features,
                     train_targets,
                     target_name=target_name)

convert_to_tfrecords(os.path.join('Dota_data','mixed_skill','dota2_test_data',
                     target_name+'dota2_test_data.tfrecords'),
                     test_features,
                     test_targets,
                     target_name=target_name)

convert_to_tfrecords(os.path.join('Dota_data','mixed_skill','dota2_validation_data',
                     target_name+'dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets,
                     target_name=target_name)


Data progress: 0/201191
Data progress: 5000/201191
Data progress: 10000/201191
Data progress: 15000/201191
Data progress: 20000/201191
Data progress: 25000/201191
Data progress: 30000/201191
Data progress: 35000/201191
Data progress: 40000/201191
Data progress: 45000/201191
Data progress: 50000/201191
Data progress: 55000/201191
Data progress: 60000/201191
Data progress: 65000/201191
Data progress: 70000/201191
Data progress: 75000/201191
Data progress: 80000/201191
Data progress: 85000/201191
Data progress: 90000/201191
Data progress: 95000/201191
Data progress: 100000/201191
Data progress: 105000/201191
Data progress: 110000/201191
Data progress: 115000/201191
Data progress: 120000/201191
Data progress: 125000/201191
Data progress: 130000/201191
Data progress: 135000/201191
Data progress: 140000/201191
Data progress: 145000/201191
Data progress: 150000/201191
Data progress: 155000/201191
Data progress: 160000/201191
Data progress: 165000/201191
Data progress: 170000/201191
Data progr

In [24]:
#create no vocab tfrecord for use in no feature column models
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
test_len = int((len(dota2_df) - train_len)/1.5)
validation_len = len(dota2_df) - train_len - test_len


target_name='radiant_win'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len:train_len+test_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len+test_len:,:])

if target_name=='radiant_win':
    train_targets = preprocess_targets_win(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_win(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_win(dota2_df.iloc[train_len+test_len:,:])
elif target_name=='gold_diff':
    train_targets = preprocess_targets_gold(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_gold(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_gold(dota2_df.iloc[train_len+test_len:,:])
elif target_name=='xp_diff':
    train_targets = preprocess_targets_xp(dota2_df.iloc[:train_len,:])
    test_targets = preprocess_targets_xp(dota2_df.iloc[train_len:train_len+test_len,:])
    validation_targets = preprocess_targets_xp(dota2_df.iloc[train_len+test_len:,:])

convert_to_tfrecords_novocab(os.path.join('Dota_data','mixed_skill','dota2_training_data',
                     target_name+'_novocab_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets,
                     target_name=target_name)

convert_to_tfrecords_novocab(os.path.join('Dota_data','mixed_skill','dota2_test_data',
                     target_name+'_novocab_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets,
                     target_name=target_name)

convert_to_tfrecords_novocab(os.path.join('Dota_data','mixed_skill','dota2_validation_data',
                     target_name+'_novocab_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets,
                     target_name=target_name)


Data progress: 0/201191
Data progress: 5000/201191
Data progress: 10000/201191
Data progress: 15000/201191
Data progress: 20000/201191
Data progress: 25000/201191
Data progress: 30000/201191
Data progress: 35000/201191
Data progress: 40000/201191
Data progress: 45000/201191
Data progress: 50000/201191
Data progress: 55000/201191
Data progress: 60000/201191
Data progress: 65000/201191
Data progress: 70000/201191
Data progress: 75000/201191
Data progress: 80000/201191
Data progress: 85000/201191
Data progress: 90000/201191
Data progress: 95000/201191
Data progress: 100000/201191
Data progress: 105000/201191
Data progress: 110000/201191
Data progress: 115000/201191
Data progress: 120000/201191
Data progress: 125000/201191
Data progress: 130000/201191
Data progress: 135000/201191
Data progress: 140000/201191
Data progress: 145000/201191
Data progress: 150000/201191
Data progress: 155000/201191
Data progress: 160000/201191
Data progress: 165000/201191
Data progress: 170000/201191
Data progr

### Make dual gold_win tfrecord files

In [21]:
#save tfrecord file with vocab needed
tier = 'mixed_skill'
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
validation_len = int((len(dota2_df) - train_len)/1.5)
test_len = len(dota2_df) - train_len - validation_len

target_name='dual_gold_win'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len:train_len+validation_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len+validation_len:,:])

train_targets = preprocess_targets_dual_gold_win(dota2_df.iloc[:train_len,:])
validation_targets = preprocess_targets_dual_gold_win(\
                                        dota2_df.iloc[train_len:train_len+validation_len,:])
test_targets = preprocess_targets_dual_gold_win(dota2_df.iloc[train_len+validation_len:,:])

convert_to_tfrecords_dual(os.path.join('dota_data',tier,'dota2_training_data', 
                     target_name+'_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets)
convert_to_tfrecords_dual(os.path.join('dota_data',tier,'dota2_test_data', 
                     target_name+'_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_dual(os.path.join('dota_data',tier,'dota2_validation_data',
                     target_name+'_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Data progress: 0/201191
Data progress: 10000/201191
Data progress: 20000/201191
Data progress: 30000/201191
Data progress: 40000/201191
Data progress: 50000/201191
Data progress: 60000/201191
Data progress: 70000/201191
Data progress: 80000/201191
Data progress: 90000/201191
Data progress: 100000/201191
Data progress: 110000/201191
Data progress: 120000/201191
Data progress: 130000/201191
Data progress: 140000/201191
Data progress: 150000/201191
Data progress: 160000/201191
Data progress: 170000/201191
Data progress: 180000/201191
Data progress: 190000/201191
Data progress: 200000/201191
heroes not in vocab 0
Data progress: 0/28742
Data progress: 10000/28742
Data progress: 20000/28742
heroes not in vocab 0
Data progress: 0/57483
Data progress: 10000/57483
Data progress: 20000/57483
Data progress: 30000/57483
Data progress: 40000/57483
Data progress: 50000/57483
heroes not in vocab 0


In [40]:
#save dual gold win tfrecord with no vocab needed
tier = 'mixed_skill'
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
validation_len = int((len(dota2_df) - train_len)/1.5)
test_len = len(dota2_df) - train_len - validation_len

target_name='dual_gold_win_novocab'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len:train_len+validation_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len+validation_len:,:])

train_targets = preprocess_targets_dual_gold_win(dota2_df.iloc[:train_len,:])
validation_targets = preprocess_targets_dual_gold_win(\
                                        dota2_df.iloc[train_len:train_len+validation_len,:])
test_targets = preprocess_targets_dual_gold_win(dota2_df.iloc[train_len+validation_len:,:])

convert_to_tfrecords_dual_novocab(os.path.join('dota_data',tier,'dota2_training_data', 
                     target_name+'_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets)
convert_to_tfrecords_dual_novocab(os.path.join('dota_data',tier,'dota2_test_data', 
                     target_name+'_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_dual_novocab(os.path.join('dota_data',tier,'dota2_validation_data',
                     target_name+'_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Data progress: 0/201191
Data progress: 10000/201191
Data progress: 20000/201191
Data progress: 30000/201191
Data progress: 40000/201191
Data progress: 50000/201191
Data progress: 60000/201191
Data progress: 70000/201191
Data progress: 80000/201191
Data progress: 90000/201191
Data progress: 100000/201191
Data progress: 110000/201191
Data progress: 120000/201191
Data progress: 130000/201191
Data progress: 140000/201191
Data progress: 150000/201191
Data progress: 160000/201191
Data progress: 170000/201191
Data progress: 180000/201191
Data progress: 190000/201191
Data progress: 200000/201191
heroes not in vocab 0
Data progress: 0/28742
Data progress: 10000/28742
Data progress: 20000/28742
heroes not in vocab 0
Data progress: 0/57483
Data progress: 10000/57483
Data progress: 20000/57483
Data progress: 30000/57483
Data progress: 40000/57483
Data progress: 50000/57483
heroes not in vocab 0


### Save triple target gold win tfrecord with no vocab

In [47]:
#save triple gold win tfrecord with no vocab needed
tier = 'mixed_skill'
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
validation_len = int((len(dota2_df) - train_len)/1.5)
test_len = len(dota2_df) - train_len - validation_len

target_name='triple_gold_win_novocab'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len:train_len+validation_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len+validation_len:,:])

train_targets = preprocess_targets_triple_gold_win(dota2_df.iloc[:train_len,:])
validation_targets = preprocess_targets_triple_gold_win(
                                        dota2_df.iloc[train_len:train_len+validation_len,:])
test_targets = preprocess_targets_triple_gold_win(dota2_df.iloc[train_len+validation_len:,:])

convert_to_tfrecords_triple_novocab(os.path.join('dota_data',tier,'dota2_training_data', 
                     target_name+'_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets)
convert_to_tfrecords_triple_novocab(os.path.join('dota_data',tier,'dota2_test_data', 
                     target_name+'_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_triple_novocab(os.path.join('dota_data',tier,'dota2_validation_data',
                     target_name+'_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Data progress: 0/201191
Data progress: 10000/201191
Data progress: 20000/201191
Data progress: 30000/201191
Data progress: 40000/201191
Data progress: 50000/201191
Data progress: 60000/201191
Data progress: 70000/201191
Data progress: 80000/201191
Data progress: 90000/201191
Data progress: 100000/201191
Data progress: 110000/201191
Data progress: 120000/201191
Data progress: 130000/201191
Data progress: 140000/201191
Data progress: 150000/201191
Data progress: 160000/201191
Data progress: 170000/201191
Data progress: 180000/201191
Data progress: 190000/201191
Data progress: 200000/201191
heroes not in vocab 0
Data progress: 0/28742
Data progress: 10000/28742
Data progress: 20000/28742
heroes not in vocab 0
Data progress: 0/57483
Data progress: 10000/57483
Data progress: 20000/57483
Data progress: 30000/57483
Data progress: 40000/57483
Data progress: 50000/57483
heroes not in vocab 0


### Save penta target gold, win, tower_damage tfrecord

In [35]:
#save penta gold win tower damage tfrecord with no vocab needed
tier = 'mixed_skill'
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
validation_len = int((len(dota2_df) - train_len)/1.5)
test_len = len(dota2_df) - train_len - validation_len

target_name='penta_gold_win_towdam_novocab'
train_features = preprocess_features(dota2_df.iloc[:train_len,:])
validation_features = preprocess_features(dota2_df.iloc[train_len:train_len+validation_len,:])
test_features = preprocess_features(dota2_df.iloc[train_len+validation_len:,:])

train_targets = preprocess_targets_penta_gold_win_towdam(dota2_df.iloc[:train_len,:])
validation_targets = preprocess_targets_penta_gold_win_towdam(
                                        dota2_df.iloc[train_len:train_len+validation_len,:])
test_targets = preprocess_targets_penta_gold_win_towdam(dota2_df.iloc[train_len+validation_len:,:])

convert_to_tfrecords_penta_novocab(os.path.join('dota_data',tier,'dota2_training_data', 
                     target_name+'_dota2_training_data.tfrecords'),
                     train_features,
                     train_targets)
convert_to_tfrecords_penta_novocab(os.path.join('dota_data',tier,'dota2_test_data', 
                     target_name+'_dota2_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords_penta_novocab(os.path.join('dota_data',tier,'dota2_validation_data',
                     target_name+'_dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)

Data progress: 0/201191
Data progress: 10000/201191
Data progress: 20000/201191
Data progress: 30000/201191
Data progress: 40000/201191
Data progress: 50000/201191
Data progress: 60000/201191
Data progress: 70000/201191
Data progress: 80000/201191
Data progress: 90000/201191
Data progress: 100000/201191
Data progress: 110000/201191
Data progress: 120000/201191
Data progress: 130000/201191
Data progress: 140000/201191
Data progress: 150000/201191
Data progress: 160000/201191
Data progress: 170000/201191
Data progress: 180000/201191
Data progress: 190000/201191
Data progress: 200000/201191
heroes not in vocab 0
Data progress: 0/28742
Data progress: 10000/28742
Data progress: 20000/28742
heroes not in vocab 0
Data progress: 0/57483
Data progress: 10000/57483
Data progress: 20000/57483
Data progress: 30000/57483
Data progress: 40000/57483
Data progress: 50000/57483
heroes not in vocab 0
